In [1]:
import numpy as np
import scipy as sp
import math as ma
import qutip as qt
import matplotlib.pyplot as plt
import miscfuncs as mf
import pyplotsetup
%matplotlib inline

In [2]:
fockdim = 200

# Square GKP code stabiliser generators
alpha = np.sqrt(0.5*np.pi)
beta = 1j*alpha 
Sx = mf.displacement_fock(2*alpha, fockdim)
Sz = mf.displacement_fock(2*beta, fockdim)

Mx, My, Mz = mf.gkp_measure_ops(fockdim, alpha, beta) # Pauli-bin operators
a = qt.destroy(fockdim) # annihilation operator

In [3]:
def squeezing_parameter(state, dB=True):
    if state.type == 'ket':
        rho = state*state.dag()
    else:
        rho = state
    Deltax = np.sqrt(-np.log(np.abs((Sx*rho).tr())**2)/(2*np.pi))
    Deltaz = np.sqrt(-np.log(np.abs((Sz*rho).tr())**2)/(2*np.pi))
    Delta = 0.5*(Deltax + Deltaz)
    if dB == True:
        return -10*np.log10(Delta**2)
    else:
        return Delta

In [4]:
def complex_quad(func, a, b, **kwargs):
    def real_func(x, *args):
        return sp.real(func(x, *args))
    def imag_func(x, *args):
        return sp.imag(func(x, *args))
    real_integral = sp.integrate.quad(real_func, a, b, **kwargs)
    imag_integral = sp.integrate.quad(imag_func, a, b, **kwargs)
    return (real_integral[0] + 1j*imag_integral[0], real_integral[1] + 1j*imag_integral[1])

In [5]:
def complex_dblquad(func, a, b, gfun, hfun, **kwargs):
    def real_func(x, *args):
        return sp.real(func(x, *args))
    def imag_func(x, *args):
        return sp.imag(func(x, *args))
    real_integral = sp.integrate.dblquad(real_func, a, b, gfun, hfun, **kwargs)
    imag_integral = sp.integrate.dblquad(imag_func, a, b, gfun, hfun, **kwargs)
    return (real_integral[0] + 1j*imag_integral[0], real_integral[1] + 1j*imag_integral[1])

In [6]:
def integrand1(theta, *args):
    m = args[0]
    n = args[1]
    A = args[2]
    def f(theta):
        f = 0
        for n in range(len(A)):
            if n == 0:
                f += 0.5*A[n]
            else: 
                f += A[n]*np.cos(4*n*theta)
        return f
    return 0.5*( mf.displacement_fock(1j*(2*ma.pi)**0.5 * np.exp(1j*theta), (m,n)) + \
                mf.displacement_fock(-1j*(2*ma.pi)**0.5 * np.exp(1j*theta), (m,n)) ) * f(theta)

In [7]:
def integrand2(theta1, theta2, *args):
    m = args[0]
    n = args[1]
    A = args[2]
    def f(theta):
        f = 0
        for n in range(len(A)):
            if n == 0:
                f += 0.5*A[n]
            else: 
                f += A[n]*np.cos(4*n*theta)
        return f
    return 0.25*2j*np.sin(2*ma.pi*np.sin(theta1 - theta2)) * \
        ( mf.displacement_fock(1j*(2*ma.pi)**0.5 * (np.exp(1j*theta1) + np.exp(1j*theta2)), (m, n)) + \
        mf.displacement_fock(-1j*(2*ma.pi)**0.5 * (np.exp(1j*theta1) + np.exp(1j*theta2)), (m, n)) - \
        mf.displacement_fock(1j*(2*ma.pi)**0.5 * (np.exp(1j*theta1) - np.exp(1j*theta2)), (m, n)) - \
        mf.displacement_fock(-1j*(2*ma.pi)**0.5 * (np.exp(1j*theta1) - np.exp(1j*theta2)), (m, n)) ) * f(theta1) * f(theta2)

# Time-averaged Hamiltonian (1st order correction)

In [8]:
N = 4
A = np.ones(N+1)
m = 0
n = 0
(res, error) = complex_quad(integrand1, 0, 2*ma.pi, args = (m,n,A))

In [9]:
J = 1e-2
H1_matelt = J/(2*ma.pi)*res
H1_error = J/(2*ma.pi)*error
Hideal_matelt = J*0.5*( mf.displacement_fock( (2*ma.pi)**0.5, (m,n)) + mf.displacement_fock( -(2*ma.pi)**0.5, (m,n)) \
              + mf.displacement_fock( 1j*(2*ma.pi)**0.5, (m,n)) + mf.displacement_fock( -1j*(2*ma.pi)**0.5, (m,n)) )
print(f'<{m}|4*H1|{n}> = {4*H1_matelt:.2e} +- {4*H1_error:.2e}')
print(f'<{m}|Hideal|{n}> = {Hideal_matelt:.2e}')

<0|4*H1|0> = 8.64e-04+0.00e+00j +- 1.02e-12+0.00e+00j
<0|Hideal|0> = 8.64e-04+0.00e+00j


# 2nd order correction

In [10]:
%%time
m = 4*(N+1)
n = 0
(res, error) = complex_dblquad(integrand2, 0, 2*ma.pi, lambda theta1: 0, lambda theta1: theta1, args = (m,n,A))

Wall time: 1min 27s


In [11]:
J = 1e-2
w = 1
H2_matelt = J**2/(4*1j*ma.pi*w) * res
H2_error = J**2/(4*1j*ma.pi*w) * error
print(f'<{m}|4*H2|{n}> = {4*H2_matelt:.2e} +- {4*H2_error:.2e}')

<20|4*H2|0> = -1.98e-06+3.11e-22j +- 4.73e-13-4.73e-13j
